In [1]:
from vyper.user import Model
from vyper.user.explorer import DataProfiler
from vyper.utils.tools import StatisticalTools as st
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from pandas.api.types import is_string_dtype
from openpyxl import Workbook
import pandas as pd
import numpy as np
import re
import statsmodels.api as sm
import math
import warnings
import string

from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.chart import BarChart, Reference, LineChart
from openpyxl.drawing.text import CharacterProperties
from openpyxl.styles import PatternFill, Border, Side, Alignment, Font

In [2]:
def Snapshot_Profile(profile_data, segment_var, segments, segment_names, include, continuous_var_bounds,
                     variable_order, other_segment = False, file = 'Profile', exclude = '',
                     PPT = True, continuous = [], excludeother = False) -> pd.DataFrame:
    profile_data = pd.DataFrame(profile_data)
    if (segments is None):
        segments = pd.unique(profile_data[segment_var])
        other_segment = False
        segment_names = str(segments)
    if (not (isinstance(other_segment, bool))):
        raise ValueError('other must be True or False.')
    if (segment_names is None):
        segment_names = str(segments)
        if (other_segment):
            segment_names = [segment_names, 'other']
    if (len(segments)+other_segment!=len(segment_names)):
        if(len(segments)==len(segment_names)):
            segment_names = [segment_names, 'other']
        else:
            warnings.warn('Incorrect length of names, replacing by segments')
            segment_names = str(segments)
        if (other_segment):
            segment_names = [segment_names, 'other']

    varclass = pd.DataFrame()
    for col in profile_data.columns:
        varclass = pd.concat([varclass.reset_index(drop=True), pd.DataFrame([st.classify_variable(profile_data[col])], columns = [col])], axis=1)

    # Continuous Variables that were classified as categorical
    varclass[varclass.columns in continuous] = 'continuous'
    if (not (include is None)):
        exclude = varclass.columns[varclass.columns not in include]
    # Variables to exclude
    varclass[varclass.columns in exclude] = 'exclude'
    if (excludeother): varclass[varclass.columns =='Categorical+other'] = 'exclude'


    continuous_var_cuts = dict()
    for variable in varclass.columns[varclass.iloc[0] == 'continuous']:
        if not continuous_var_bounds:
            continuous_var_bounds = dict()
        if variable in continuous_var_bounds:
            continuous_var_cuts[variable] = continuous_var_bounds[variable]
        else:
            continuous_var_cuts[variable] = st.get_breaks(profile_data.loc[:, variable], nbins=10,
                     squash_extremes=False)

    for i in range(len(segments)):
        profile_1 = DataProfiler(profile_data[profile_data[segment_var] == segments[i]]).create_profile(number_of_bins = 10,
                                                                          cts_cuts = continuous_var_cuts)
        if (i == 0):
            profile = profile_1
        else:
            profile = pd.merge(profile, profile_1, on = ['Variable', 'Category'], how = 'outer')

    for i in profile.columns:
        if ('Count' in i):
            profile[i][profile[i].isna()] = 0
        if ('Percent' in i):
            profile[i][profile[i].isna()] = 0
            
            
    if file:
        import os
        # Make Profile ####
        filesave = file + '.xlsx'
        
        if os.path.exists(filesave):
            os.remove(filesave)
        CreateXLProfile_Snap(profile, savepath = filesave)
        
        if PPT:
            wb = Workbook()

            profile['Category'] = profile['Category'].astype(str)

            all_var_profiling_ws = wb.create_sheet(profile['Variable'].iloc[0], 0)
            # make new sheet the active sheet we are working on
            all_var_profiling_ws = wb.active

            # remove gridlines from the sheet
            all_var_profiling_ws.sheet_view.showGridLines = False

            # set border line thickness
            thick = Side(border_style="thick", color="000000")
            # medium = Side(border_style="medium", color="000000")
            thin = Side(border_style="thin", color="000000")

            # get max rows and cols of profiling df
            max_rows = profile.shape[0]
            max_cols = profile.shape[1]
            
            input_cols = list(string.ascii_uppercase)

            j = 4
            for z in range(0, len(profile)):
                if (z != 0):
                    if (profile['Variable'].iloc[z] != profile['Variable'].iloc[z - 1]):
                        # create sheet
                        all_var_profiling_ws = wb.create_sheet(profile['Variable'].iloc[z], 0)
                        # make new sheet the active sheet we are working on
                        all_var_profiling_ws = wb.active

                        for y in range(len(profile.columns)):
                            current_cell = all_var_profiling_ws.cell(row = 3, column = y + 2)
                            current_cell.font = Font(bold=True)
                            current_cell.alignment = Alignment(horizontal='center', vertical='center')
                            current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
                            current_cell.value = profile.columns[y]
                            if y == 0:
                                current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
                            elif y == max_cols - 1:
                                current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
                            else:
                                current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)
                        j = 4
                    else:
                        incol = 2
                        for y in range(len(profile.columns)):
                            current_cell = all_var_profiling_ws.cell(row = 3, column = incol)
                            incol = incol + 1
                            current_cell.font = Font(bold=True)
                            current_cell.alignment = Alignment(horizontal='center', vertical='center')
                            current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
                            current_cell.value = profile.columns[y]
                            if y == 0:
                                current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
                            elif y == max_cols - 1:
                                current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
                            else:
                                current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)
                            if (y % 2 == 1 and y > 2):
                                current_cell = all_var_profiling_ws.cell(row = 3, column = incol)
                                incol = incol + 1
                                current_cell.font = Font(bold=True)
                                current_cell.alignment = Alignment(horizontal='center', vertical='center')
                                current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
                                current_cell.value = 'Index'
                                if y == max_cols - 1:
                                    current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
                                else:
                                    current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)
                                
                incol = 2
                for y in range(len(profile.columns)):
                    current_cell = all_var_profiling_ws.cell(row = j, column = incol)
                    incol = incol + 1
                    current_cell.value = profile.iloc[z, y]
                    if y == 0:
                        current_cell.border = Border(top=thin, left=thick, right=thin, bottom=thin)
                    else:
                        current_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
                    if (y % 2 == 1 and y > 2):
                        current_cell = all_var_profiling_ws.cell(row = j, column = incol)
                        incol = incol + 1
                        current_cell.value = '=' + input_cols[incol - 3] + str(j) + '/E' + str(j)
                        if y == max_cols - 1:
                            current_cell.border = Border(top=thin, left=thin, right=thick, bottom=thin)
                        else:
                            current_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)

                j = j + 1
            
            filesave = file + 'PPT.xlsx'
        
            if os.path.exists(filesave):
                os.remove(filesave)
            
            wb.save(filesave)
            
    return profile;

In [3]:
def CreateXLProfile_Snap(profile, savepath):
    wb = Workbook()

    profile['Category'] = profile['Category'].astype(str)

    # create sheet
    all_var_profiling_ws = wb.create_sheet('profile', 0)

    # make new sheet the active sheet we are working on
    all_var_profiling_ws = wb.active

    # remove gridlines from the sheet
    all_var_profiling_ws.sheet_view.showGridLines = False

    # set border line thickness
    thick = Side(border_style="thick", color="000000")
    # medium = Side(border_style="medium", color="000000")
    thin = Side(border_style="thin", color="000000")

    # get max rows and cols of profiling df
    max_rows = profile.shape[0]
    max_cols = profile.shape[1]

    input_rows = range(4, max_rows + 4)
    input_cols = list(string.ascii_uppercase)[1:max_cols + 1]

    y = 0
    for i in input_cols:
        x = "3"
        x = i + x

        all_var_profiling_ws[x].font = Font(bold=True)
        current_cell = all_var_profiling_ws[x]
        current_cell.alignment = Alignment(horizontal='center', vertical='center')
        current_cell.fill = PatternFill("solid", fgColor="A9C4FE")

        if y == 0:
            current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
        elif y == max_cols - 1:
            current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
        else:
            current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)

        all_var_profiling_ws[x] = profile.columns[y]
        y = y + 1

    y = 0
    for i in input_cols:
        z = 0
        for j in input_rows:
            all_var_profiling_ws.row_dimensions[j].height = 25
            x = str(j)
            x = i + x
            # print(x)
            current_cell = all_var_profiling_ws[x]
            current_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)

            if profile['Variable'].iloc[z] != profile['Variable'].iloc[z-1]:
                current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)

            all_var_profiling_ws[x] = profile.iloc[z, y]

            if y == 0:
                current_cell.border = Border(top=thin, left=thick, right=thin, bottom=thin)
                if profile['Variable'].iloc[z] != profile['Variable'].iloc[z-1]:
                    current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
            elif y == max_cols - 1:
                current_cell.border = Border(top=thin, left=thin, right=thick, bottom=thin)
                if profile['Variable'].iloc[z] != profile['Variable'].iloc[z-1]:
                    current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)


            z = z + 1
        y = y + 1

    if (savepath != ''):
        wb.save(savepath)

In [6]:
eps_data = pd.read_csv(r"Data/selected_attributes.csv")

In [7]:
eps_data['seg'] = np.random.randint(0, 3, eps_data.shape[0])

In [8]:
profile = Snapshot_Profile(profile_data = eps_data, segment_var='seg', segments = None, segment_names = None, include = None,
                 continuous_var_bounds = None, variable_order = None, other_segment = False,
                 file = 'Profile', exclude = [], PPT = True, continuous = [], excludeother = False)
profile

ValueError: Invalid character : found in sheet title